# Text classification of news headlines for prediction of news category

# 1) Independent Variable: 'title', Target: 'frequent_category'

Our dataset contains many news entries with missing category (unlabeled data). We now wan't to train a classification model with our labeled data (that is the news entries with category label provided). This model then will be used to predict the category labels for the unlabeled part of the dataset.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics

## Load train and test set

In [2]:
train = pd.read_csv('../../preprocessing/train.csv')
test = pd.read_csv('../../preprocessing/test.csv')

In [3]:
train.head()

,date,title,description,category,text,source,frequent_category
0,2023-06-29 11:57:00,Cyberbetrug in Philippinen: Großrazzia gegen M...,In den Philippinen wurden 2.724 Personen festg...,Missing,NaN,taz,Missing
1,2023-01-15 09:16:00,"GPT: Eine Software, die die Welt verändert","GPT-3 ist eine jener Entwicklungen, die binnen...",Terra X - die Wissens-Kolumne,NaN,ZDF heute,Other
2,2023-03-04 15:13:32,Fußball: Hollerbach schießt Wehen Wiesbaden mi...,NaN,News,NaN,Zeit,News
3,2022-07-16 07:17:44,Leichtathletik-WM: Ein Auftakt mit Licht und S...,"Stabhochspringerin Jacqueline Otchere jubelte,...",Missing,NaN,Tagesschau,Missing
4,2022-10-11 18:49:00,Geflüchtete in Deutschland: Nicht alle gleich ...,Die Innenministerin spricht sich für Solidarit...,Missing,NaN,taz,Missing


In [4]:
test.head()

,date,title,description,category,text,source,frequent_category
0,2022-07-15 09:40:00,Lauterbach für vierte Corona-Impfung: Zweiter ...,Wegen steigender Coronazahlen rät Gesundheitsm...,Missing,NaN,taz,Missing
1,2023-06-15 22:08:22,Weyrauch zieht Bewerbung um rbb-Intendantenste...,Einen Tag vor der Intendantenwahl beim rbb hat...,Missing,NaN,Tagesschau,Missing
2,2022-08-20 06:29:28,"Inflation in Großbritannien: ""Es wird alles te...",Die Energiepreise: steigen und steigen. Die In...,Missing,NaN,Tagesschau,Missing
3,2023-02-12 22:14:58,Nahe kanadischer Grenze - Schon wieder! USA sc...,Medienberichten zufolge haben die USA erneut e...,Ausland,NaN,Focus,Ausland
4,2022-08-26 11:50:37,China: Geschichte verdeht: Mao wollte Taiwan a...,Chinas Kommunistische Partei bedroht Taiwan mi...,"China, Taiwan, Kommunistische Partei, Peking, ...",NaN,stern,Other


## Examine independent variable title

In [5]:
print(train['title'].isnull().sum())

0


In [6]:
print(test['title'].isnull().sum())

0


## Examine target variable frequent_category

In [7]:
print(train.groupby('frequent_category')['frequent_category'].count() / len(train))

frequent_category
Ausland          0.055276
Deutschland      0.048290
Missing          0.382115
News             0.059685
Other            0.407577
Politik          0.011713
Ukraine-Krise    0.018494
Wirtschaft       0.016850
Name: frequent_category, dtype: float64


In [8]:
print(test.groupby('frequent_category')['frequent_category'].count() / len(test))

frequent_category
Ausland          0.055369
Deutschland      0.054771
Missing          0.373758
News             0.060001
Other            0.410670
Politik          0.013151
Ukraine-Krise    0.016962
Wirtschaft       0.015318
Name: frequent_category, dtype: float64


We have many observations which are not labeled. We can't use them for training the classification model but we can use them as 'real' input for predicting category labels.

In [9]:
unlabeled = pd.concat([train[train['frequent_category'] == 'Missing'], test[test['frequent_category'] == 'Missing']])
unlabeled.head()

,date,title,description,category,text,source,frequent_category
0,2023-06-29 11:57:00,Cyberbetrug in Philippinen: Großrazzia gegen M...,In den Philippinen wurden 2.724 Personen festg...,Missing,NaN,taz,Missing
3,2022-07-16 07:17:44,Leichtathletik-WM: Ein Auftakt mit Licht und S...,"Stabhochspringerin Jacqueline Otchere jubelte,...",Missing,NaN,Tagesschau,Missing
4,2022-10-11 18:49:00,Geflüchtete in Deutschland: Nicht alle gleich ...,Die Innenministerin spricht sich für Solidarit...,Missing,NaN,taz,Missing
10,2022-06-03 08:55:08,Gebrauchtwagen im Handel fast ein Drittel teurer,Lieferengpässe und Materialmangel in der Autob...,Missing,NaN,Tagesschau,Missing
11,2022-07-08 18:48:00,Großbritanniens Noch-Premier Johnson: Lahme En...,Boris Johnson nahm es mit Fakten nie so genau ...,Missing,NaN,taz,Missing


In [10]:
print(unlabeled.groupby('frequent_category')['frequent_category'].count() / len(unlabeled))

frequent_category
Missing    1.0
Name: frequent_category, dtype: float64


In [11]:
train = train[train['frequent_category'] != 'Missing']

In [12]:
print(train.groupby('frequent_category')['frequent_category'].count() / len(train))

frequent_category
Ausland          0.089461
Deutschland      0.078153
News             0.096596
Other            0.659632
Politik          0.018956
Ukraine-Krise    0.029931
Wirtschaft       0.027271
Name: frequent_category, dtype: float64


In [13]:
test = test[test['frequent_category'] != 'Missing']

In [14]:
print(test.groupby('frequent_category')['frequent_category'].count() / len(test))

frequent_category
Ausland          0.088414
Deutschland      0.087460
News             0.095812
Other            0.655769
Politik          0.021000
Ukraine-Krise    0.027085
Wirtschaft       0.024460
Name: frequent_category, dtype: float64


## Create input and output for classification

In [15]:
X_train = train['title']
y_train = train['frequent_category']

In [16]:
print(X_train)
print(y_train)

1               GPT: Eine Software, die die Welt verändert
2        Fußball: Hollerbach schießt Wehen Wiesbaden mi...
5        Pariser Terrorprozess: Staatsanwaltschaft ford...
6        „Frust ist gewaltig“ - US-Medien zerlegen Deut...
7               USA und London kündigen neue Sanktionen an
                               ...                        
53525                  Roger Federer, der König der Herzen
53526    Kriminalität: Museum verschiebt nach Golddiebs...
53528    1,3 Milliarden Menschen, eine Milliarde Überwa...
53529    Europäische Union: EVP-Chef: "Europa de facto ...
53530                Obama über Beliebtheit in der Familie
Name: title, Length: 33076, dtype: object
1                Other
2                 News
5              Ausland
6        Ukraine-Krise
7                Other
             ...      
53525            Other
53526             News
53528            Other
53529            Other
53530            Other
Name: frequent_category, Length: 33076, dtype: object


In [17]:
X_test = test['title']
y_test = test['frequent_category']

In [18]:
print(X_test)
print(y_test)

3        Nahe kanadischer Grenze - Schon wieder! USA sc...
4        China: Geschichte verdeht: Mao wollte Taiwan a...
8             Klimakrise: Keine Zeit mehr zum Ausprobieren
10                     Das Rätsel vom purpurfarbenen Teich
11                      DFB stellt Weichen für die Zukunft
                               ...                        
13374    USA: Biden wirbt für Corona-Impfung für Unter-...
13377    Kommentar von Ulrich Reitz - Keinen „Fuchs“ fü...
13378    Jan Böhmermann und das „Ibiza-Video“ – Raunen ...
13379    „Kann mein ganzes Agieren nicht nach fünf Proz...
13381               "Krieg hätte verhindert werden können"
Name: title, Length: 8381, dtype: object
3            Ausland
4              Other
8              Other
10             Other
11             Other
            ...     
13374          Other
13377    Deutschland
13378    Deutschland
13379    Deutschland
13381          Other
Name: frequent_category, Length: 8381, dtype: object


In [19]:
X_unlabeled = unlabeled['title']

In [20]:
print(X_unlabeled)

0        Cyberbetrug in Philippinen: Großrazzia gegen M...
3        Leichtathletik-WM: Ein Auftakt mit Licht und S...
4        Geflüchtete in Deutschland: Nicht alle gleich ...
10        Gebrauchtwagen im Handel fast ein Drittel teurer
11       Großbritanniens Noch-Premier Johnson: Lahme En...
                               ...                        
13371    Mindestens fünf Tote durch Hurrikan in der Kar...
13375    Sozialarbeiter über Wohnungslosigkeit: „Es geh...
13376    G7-Gipfel in Elmau endet - Weiterreise zum NAT...
13380    Klimabewegung und Grüne: Lützerath als Zerreiß...
13382    EZB-Entscheid: Welcher Leitzins für Sparer wic...
Name: title, Length: 25457, dtype: object


### Perform featured extraction out of X_train text data
(extract terms and term frequency)

In [21]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

In [22]:
print(X_train_tfidf.shape)  #(title entries, different words over all titles)

(33076, 35376)


## Train a Logistic Regression Classifier for prediction of the category label

In [23]:
clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Create Pipeline

In [24]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), 
                     ('clf', LogisticRegression())])
text_clf.fit(X_train, y_train)

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LogisticRegression())])

## Predict category label for test set

In [25]:
predictions = text_clf.predict(X_test)

In [26]:
print(metrics.confusion_matrix(y_test, predictions))

[[ 216    6    2  504    0   13    0]
 [   3   99    6  622    0    1    2]
 [   2    0  289  512    0    0    0]
 [  54   61   49 5303    0   18   11]
 [   6   18    0  138   11    3    0]
 [  10    0    0  168    0   49    0]
 [   0    6    0  197    0    0    2]]


In [27]:
print(metrics.classification_report(y_test, predictions))

               precision    recall  f1-score   support

      Ausland       0.74      0.29      0.42       741
  Deutschland       0.52      0.14      0.21       733
         News       0.84      0.36      0.50       803
        Other       0.71      0.96      0.82      5496
      Politik       1.00      0.06      0.12       176
Ukraine-Krise       0.58      0.22      0.32       227
   Wirtschaft       0.13      0.01      0.02       205

     accuracy                           0.71      8381
    macro avg       0.65      0.29      0.34      8381
 weighted avg       0.70      0.71      0.65      8381



In [28]:
print(metrics.accuracy_score(y_test, predictions))

0.7122061806467008


In [29]:
print(predictions)
print(predictions.shape)

['Other' 'Other' 'Other' ... 'Other' 'Other' 'Other']
(8381,)


In [30]:
test['predicted_category'] = predictions

In [31]:
test.head(20)

,date,title,description,category,text,source,frequent_category,predicted_category
3,2023-02-12 22:14:58,Nahe kanadischer Grenze - Schon wieder! USA sc...,Medienberichten zufolge haben die USA erneut e...,Ausland,NaN,Focus,Ausland,Other
4,2022-08-26 11:50:37,China: Geschichte verdeht: Mao wollte Taiwan a...,Chinas Kommunistische Partei bedroht Taiwan mi...,"China, Taiwan, Kommunistische Partei, Peking, ...",NaN,stern,Other,Other
8,2022-07-28 16:32:00,Klimakrise: Keine Zeit mehr zum Ausprobieren,Die Klimafolgen werden zunehmend drastischer. ...,Erdüberlastungstag,NaN,ZDF heute,Other,Other
10,2023-01-12 04:55:23,Das Rätsel vom purpurfarbenen Teich,Ob das auch der Klimawandel war? Wohl kaum. Ei...,Wissenschaft,NaN,Welt,Other,Other
11,2023-08-22 05:30:00,DFB stellt Weichen für die Zukunft,Ex-Bundesligatrainer Hannes Wolf wurde beim De...,Nachrichten | ZDF-Morgenmagazin,NaN,ZDF heute,Other,Other
12,2022-11-15 09:46:33,Ukraine-Krieg - Stimmen und Entwicklungen - „D...,Für einen Militärökonomen aus Zürich ist der K...,Ausland,NaN,Focus,Ausland,Ausland
13,2023-04-13 15:57:00,Tennessee: Posse um ausgeschlossene Abgeordnet...,In Tennessee sind zwei Abgeordnete wieder ins ...,"USA, Tennessee, Republikaner, Parlamentskammer...",NaN,stern,Other,Other
14,2022-07-30 17:06:00,Bayreuth: Die ewigen Spielregeln,Auf dem Hügel gelten klare Regeln - hier werde...,Kultur,NaN,Welt,Other,Other
15,2023-02-18 17:25:00,FC Bayern verliert in Unterzahl in Gladbach,Dem FC Bayern droht der Verlust der Tabellenfü...,Union Berlin winkt Platz eins,NaN,ZDF heute,Other,Other
16,2023-01-12 15:00:00,"Borna, Hildburghausen, Görlitz - was ist an Si...",Nach dem Jahreswechsel hatte SPD-Chef Klingbei...,Deutschland,NaN,Welt,Deutschland,Other


## Predict category label for unlabeled data

In [32]:
predictions_unlabeled = text_clf.predict(X_unlabeled)

In [33]:
print(predictions_unlabeled)
predictions_unlabeled.shape

['Other' 'Other' 'Other' ... 'Other' 'Other' 'Other']


(25457,)

In [34]:
unlabeled['predicted_category'] = predictions_unlabeled

In [35]:
unlabeled.head(20)

,date,title,description,category,text,source,frequent_category,predicted_category
0,2023-06-29 11:57:00,Cyberbetrug in Philippinen: Großrazzia gegen M...,In den Philippinen wurden 2.724 Personen festg...,Missing,NaN,taz,Missing,Other
3,2022-07-16 07:17:44,Leichtathletik-WM: Ein Auftakt mit Licht und S...,"Stabhochspringerin Jacqueline Otchere jubelte,...",Missing,NaN,Tagesschau,Missing,Other
4,2022-10-11 18:49:00,Geflüchtete in Deutschland: Nicht alle gleich ...,Die Innenministerin spricht sich für Solidarit...,Missing,NaN,taz,Missing,Other
10,2022-06-03 08:55:08,Gebrauchtwagen im Handel fast ein Drittel teurer,Lieferengpässe und Materialmangel in der Autob...,Missing,NaN,Tagesschau,Missing,Other
11,2022-07-08 18:48:00,Großbritanniens Noch-Premier Johnson: Lahme En...,Boris Johnson nahm es mit Fakten nie so genau ...,Missing,NaN,taz,Missing,Other
14,2023-05-21 16:14:00,Diversitätsbeauftragte im Museum: „Frauen nich...,Weiqi Wang ist Diversitätsbeauftragte in einem...,Missing,NaN,taz,Missing,Other
16,2023-07-07 08:47:34,Studie belegt wirksame Behandlung: Alzheimer-M...,Die US-amerikanische Gesundheitsbehörde FDA ha...,Missing,NaN,Tagesschau,Missing,Other
20,2023-04-23 18:57:00,SPD Berlin stimmt für Schwarz-Rot: Jetzt kommt...,Eine Mehrheit ist eine Mehrheit: Die SPD-inter...,Missing,NaN,taz,Missing,Other
21,2022-10-15 10:42:19,Britischer Finanzminister Hunt kündigt höhere ...,Der neue britische Finanzminister Hunt stimmt ...,Missing,NaN,Tagesschau,Missing,Other
23,2023-03-08 19:51:56,Greensand: Dänemark startet CO2-Speicherung im...,Bis 2030 sollen in der dänischen Nordsee bis z...,Missing,NaN,Tagesschau,Missing,Other


In [36]:
print(unlabeled.groupby('predicted_category')['predicted_category'].count())

predicted_category
Ausland            253
Deutschland        221
News               265
Other            24663
Ukraine-Krise       43
Wirtschaft          12
Name: predicted_category, dtype: int64


In [37]:
## Save result to csv
test.to_csv('predicted_cat_test_logistic_reg.csv')
unlabeled.to_csv('predicted_cat_unlabeled_logistic_reg.csv')